<a href="https://colab.research.google.com/github/tiff03/Plant-Growth-and-Water-Usage-Data-Analysis/blob/main/I_Simulated_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulated Data
---
Taking in simulated raw data and cleaning it so it outputs water usage per day

plant name: length_I, length_D, length_M, length_L


tomato: 15, 30, 28, 17




In [1]:
import numpy as np
from matplotlib import pyplot as plt
import time
from time import time, ctime
from time import struct_time
import datetime
import random
import pandas as pd
from tabulate import tabulate
import csv
from csv import writer
from google.colab import drive

path = '/content/drive/My Drive/SUMMER 22/intern/I_data/I_simulated_data.csv'


tomato_stage_data = [15, 30, 28, 17]
head = ["time stamp", "event", "sensor ID", "water reserve (mm)", "volume (mm)"]

Stage I (Initial)

In [2]:
minute_interval = 5 #how many minutes between each sensor read
sensor_ID = "1"
water_reserve = 10 #how much water initially in the reserve
days_since_water = 2 #days since plant has been watered - set at 2 so it waters first day
irrigation_volume = 0 
IR = 0
watered = False
water_time = 9 #what time to water the plants at
all_data = []
IR_data_point = []


today = datetime.datetime(2022, 5, 19)

end_time = today

#for stage in range(len(tomato_stage_data)):   account for all stages
  #for t in range(tomato_stage_data[stage]):

#elapse time every 5 minutes
for t in range(tomato_stage_data[0]):       #only I stage right now
  
  watered = False

  for h in range(24):
    time_now = end_time.time() 

    if(time_now.hour % 6 == 0):
      water_reserve -= 0.25

    for m in range(60//minute_interval):

      if(not watered and (time_now.hour == water_time and time_now.minute == 0)):
        irrigation_volume = random.randint(1,3)
        water_reserve += irrigation_volume
        watered = True
        IR_data_point = [end_time.strftime("%m/%d/%Y %H:%M:%S"), "IR", None, None, irrigation_volume]
        all_data.append(IR_data_point)

      data_point = [end_time.strftime("%m/%d/%Y %H:%M:%S"), "sensor", sensor_ID, water_reserve, None]
      all_data.append(data_point)
      end_time += datetime.timedelta(minutes = minute_interval)
      
      irrigation_volume = 0
      

with open(path, 'w') as file:
  writer = csv.writer(file)

  writer.writerow(head)
  
  for point in all_data:
    writer.writerow(point)

In [4]:
dataset = pd.read_csv(path)

dataset["sensor ID"] = dataset["sensor ID"].astype(object)

#raw simulated dataFrame
dataset

,time stamp,event,sensor ID,water reserve (mm),volume (mm)
0,05/19/2022 00:00:00,sensor,1.0,9.75,NaN
1,05/19/2022 00:05:00,sensor,1.0,9.75,NaN
2,05/19/2022 00:10:00,sensor,1.0,9.75,NaN
3,05/19/2022 00:15:00,sensor,1.0,9.75,NaN
4,05/19/2022 00:20:00,sensor,1.0,9.75,NaN
...,...,...,...,...,...
4330,06/02/2022 23:35:00,sensor,1.0,33.00,NaN
4331,06/02/2022 23:40:00,sensor,1.0,33.00,NaN
4332,06/02/2022 23:45:00,sensor,1.0,33.00,NaN
4333,06/02/2022 23:50:00,sensor,1.0,33.00,NaN


In [5]:
#create two different dataFrames one of the Irrigation event and the other of the sensor event

numeric_columns = list(dataset.select_dtypes(include=['float64']).columns)
def_columns = list(dataset.select_dtypes(include=['object']).columns)

print('Numerical Columns: ',numeric_columns)
print('Defining Columns: ',def_columns)

irrigation = dataset[dataset['event'] == 'IR']
sensor = dataset[dataset['event'] == 'sensor']

irrigation = irrigation.set_index(pd.Index(list(range(len(irrigation)))))
sensor = sensor.set_index(pd.Index(list(range(len(sensor)))))

print('irrigation: ', irrigation)
print('sensor: ', sensor)

Numerical Columns:  ['water reserve (mm)', 'volume (mm)']
Defining Columns:  ['time stamp', 'event', 'sensor ID']
irrigation:               time stamp event sensor ID  water reserve (mm)  volume (mm)
0   05/19/2022 09:00:00    IR       NaN                 NaN          3.0
1   05/20/2022 09:00:00    IR       NaN                 NaN          3.0
2   05/21/2022 09:00:00    IR       NaN                 NaN          3.0
3   05/22/2022 09:00:00    IR       NaN                 NaN          3.0
4   05/23/2022 09:00:00    IR       NaN                 NaN          1.0
5   05/24/2022 09:00:00    IR       NaN                 NaN          3.0
6   05/25/2022 09:00:00    IR       NaN                 NaN          3.0
7   05/26/2022 09:00:00    IR       NaN                 NaN          2.0
8   05/27/2022 09:00:00    IR       NaN                 NaN          1.0
9   05/28/2022 09:00:00    IR       NaN                 NaN          3.0
10  05/29/2022 09:00:00    IR       NaN                 NaN          3

Graph for Stage I


---


usage per 5 minutes

In [6]:
time_x = []
usage_y = []
previous_point_index = None
ir = 0
x = 0

for i in range(len(dataset['time stamp'])):
  if(dataset['event'][i] == 'IR'):
    ir = dataset['volume (mm)'][i]

  if(dataset['water reserve (mm)'][i] != None and previous_point_index != None): 
    usage_y.append(dataset['water reserve (mm)'][i] - dataset['water reserve (mm)'][previous_point_index] - ir)
    time_x.append(dataset['time stamp'][i])
    ir = 0
    x += 1
    
  previous_point_index = i
  
x = [i for i in range(x)]

plt.figure(figsize=(150,100))
plt.title("Water Usage over Time for Stage I", fontsize=300)
plt.xlabel('Time', fontsize=100)
plt.ylabel('Water Usage (mm)', fontsize=100)

plt.tick_params(axis='x', labelsize=80)
plt.tick_params(axis='y', labelsize=80)

default_x_ticks = range(len(x))
plt.xticks(default_x_ticks, time_x, fontsize = 80)
plt.scatter(default_x_ticks, usage_y, s = 1000)


Graph for Stage I


---
usage per day



In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import MaxNLocator
from scipy import stats

time_x = []
usage_y = []
IR_index = 0
interval = 24
start_time = '05/19/2022 00:00:00'
first = 0
second = 0

#find the start time
first = sensor.loc[sensor['time stamp'] == start_time] #dataframe object
first_index = int(sensor[sensor['time stamp']== first['time stamp'][0]].index.values)
first_water_reserve = 0
second_water_reserve = first['water reserve (mm)'][0]

for i in range(first_index, len(sensor), interval * 60 // 5):
  if(first_water_reserve != 0):
    second_water_reserve = sensor['water reserve (mm)'][i]
    IR = irrigation['volume (mm)'][IR_index]
    value = second_water_reserve - first_water_reserve - IR
    usage_y.append(value)
    time_x.append(sensor['time stamp'][i])
    IR_index+=1

  first_water_reserve = second_water_reserve


plt.figure(figsize=(150,100))
plt.title("Water Usage over Time for Stage I", fontsize=300)
plt.xlabel('Time', fontsize=100)
plt.ylabel('Water Usage (mm)', fontsize=100)

plt.tick_params(axis='x', labelsize=80)
plt.tick_params(axis='y', labelsize=80)


#plt.xticks(default_x_ticks, time_x, fontsize = 80)
default_x_ticks = range(len(time_x))

slope, intercept, r_value, p_value, std_err = stats.linregress(default_x_ticks, usage_y)

line = slope * default_x_ticks + intercept
plt.plot(default_x_ticks, line, 'r', linewidth=10, label='y={:.2f}x+{:.2f}'.format(slope,intercept))

plt.xticks(default_x_ticks, time_x, fontsize = 80)
plt.scatter(default_x_ticks, usage_y, s = 1000)
plt.legend(fontsize=100)
plt.show()

Create new cleaned dataFrame


---

data of and then water usage for that day (24 hours)



In [12]:
d = {'time stamp': time_x, 'water usage per day (mm)': usage_y}

clean_dataset = pd.DataFrame(d)

clean_dataset.to_csv('I_clean_dataset.csv')  

clean_dataset

,time stamp,water usage per day (mm)
0,05/20/2022 00:00:00,-1.0
1,05/21/2022 00:00:00,-1.0
2,05/22/2022 00:00:00,-1.0
3,05/23/2022 00:00:00,-1.0
4,05/24/2022 00:00:00,-1.0
5,05/25/2022 00:00:00,-1.0
6,05/26/2022 00:00:00,-1.0
7,05/27/2022 00:00:00,-1.0
8,05/28/2022 00:00:00,-1.0
9,05/29/2022 00:00:00,-1.0


Linear Regression attempt

In [23]:
from sklearn.linear_model import LinearRegression

default_x_ticks = np.array(default_x_ticks)
default_x_ticks = default_x_ticks.reshape(-1,1)

model = LinearRegression()

model.fit(default_x_ticks, usage_y)

r_sq = model.score(default_x_ticks, usage_y)
print(f"coefficient of determination: {r_sq}")

print(f"intercept: {model.intercept_}")

print(f"slope: {model.coef_}")

usage_y = np.array(usage_y)

new_model = LinearRegression().fit(default_x_ticks, usage_y.reshape((-1, 1)))
print(f"intercept: {new_model.intercept_}")

print(f"slope: {new_model.coef_}")


y_pred = model.predict(default_x_ticks)
print(f"predicted response:\n{y_pred}")
y_pred = model.intercept_ + model.coef_ * default_x_ticks
print(f"predicted response:\n{y_pred}")

x_new = np.arange(5).reshape((-1, 1))
print(x_new)


y_new = model.predict(x_new)
print(y_new)

coefficient of determination: 1.0
intercept: -1.0
slope: [0.]
intercept: [-1.]
slope: [[0.]]
predicted response:
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
predicted response:
[[-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]]
[[0]
 [1]
 [2]
 [3]
 [4]]
[-1. -1. -1. -1. -1.]


Adding noise to previous data

---



In [34]:
mu, sigma = 0, 1.1 
number_of_rows = len(dataset)

noise = np.around(np.random.normal(mu, sigma, [number_of_rows, 1]), 2)

for i in range(len(dataset['water reserve (mm)'])):
  if(dataset['water reserve (mm)'][i] != np.nan):
    if(noise[i] > 0):
      dataset['water reserve (mm)'][i] -= noise[i]

    if(noise[i] < 0):
      dataset['water reserve (mm)'][i] += noise[i]

  else:
    break


irrigation = dataset[dataset['event'] == 'IR']
sensor = dataset[dataset['event'] == 'sensor']

irrigation = irrigation.set_index(pd.Index(list(range(len(irrigation)))))
sensor = sensor.set_index(pd.Index(list(range(len(sensor)))))

print('irrigation: ', irrigation)
print('sensor: ', sensor)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


irrigation:               time stamp event sensor ID  water reserve (mm)  volume (mm)
0   05/19/2022 09:00:00    IR       NaN                 NaN          1.0
1   05/20/2022 09:00:00    IR       NaN                 NaN          3.0
2   05/21/2022 09:00:00    IR       NaN                 NaN          1.0
3   05/22/2022 09:00:00    IR       NaN                 NaN          1.0
4   05/23/2022 09:00:00    IR       NaN                 NaN          1.0
5   05/24/2022 09:00:00    IR       NaN                 NaN          1.0
6   05/25/2022 09:00:00    IR       NaN                 NaN          1.0
7   05/26/2022 09:00:00    IR       NaN                 NaN          3.0
8   05/27/2022 09:00:00    IR       NaN                 NaN          2.0
9   05/28/2022 09:00:00    IR       NaN                 NaN          3.0
10  05/29/2022 09:00:00    IR       NaN                 NaN          3.0
11  05/30/2022 09:00:00    IR       NaN                 NaN          3.0
12  05/31/2022 09:00:00    IR       Na

In [35]:
time_x = []
usage_y = []
IR_index = 0
interval = 24
start_time = '05/19/2022 00:00:00'
first = 0
second = 0

#find the start time
first = sensor.loc[sensor['time stamp'] == start_time] #dataframe object
first_index = int(sensor[sensor['time stamp']== first['time stamp'][0]].index.values)
first_water_reserve = 0
second_water_reserve = first['water reserve (mm)'][0]

for i in range(first_index, len(sensor), interval * 60 // 5):
  if(first_water_reserve != 0):
    second_water_reserve = sensor['water reserve (mm)'][i]
    IR = irrigation['volume (mm)'][IR_index]
    value = second_water_reserve - first_water_reserve - IR
    usage_y.append(value)
    time_x.append(sensor['time stamp'][i])
    IR_index+=1

  first_water_reserve = second_water_reserve


plt.figure(figsize=(150,100))
plt.title("Water Usage over Time for Stage I", fontsize=300)
plt.xlabel('Time', fontsize=100)
plt.ylabel('Water Usage (mm)', fontsize=100)

plt.tick_params(axis='x', labelsize=80)
plt.tick_params(axis='y', labelsize=80)


#plt.xticks(default_x_ticks, time_x, fontsize = 80)
default_x_ticks = range(len(time_x))
plt.xticks(default_x_ticks, time_x, fontsize = 80)
plt.scatter(default_x_ticks, usage_y, s = 1000)